In [1]:
import os
import cv2
import cvzone
from cvzone.ClassificationModule import Classifier

# Initialize classifier

In [2]:
classifier = Classifier('Resources/Model/keras_model.h5', 'Resources/Model/labels.txt')

# Load waste and bin images
imgWasteList = [cv2.imread(os.path.join('Resources/Waste', path), cv2.IMREAD_UNCHANGED) for path in os.listdir('Resources/Waste')]
imgBinsList = [cv2.imread(os.path.join('Resources/Bins', path), cv2.IMREAD_UNCHANGED) for path in os.listdir('Resources/Bins')]

# Mapping of waste class to bin class
classDic = {0: None, 1: 0, 2: 0, 3: 3, 4: 3, 5: 1, 6: 1, 7: 2, 8: 2}

# Main loop
cap = cv2.VideoCapture(0)  # Initialize video capture object
imgArrow = cv2.imread('Resources/arrow.png', cv2.IMREAD_UNCHANGED)  # Load arrow image


In [ ]:
while True:
    _, img = cap.read() # Capture a frame from the video
    imgResize = cv2.resize(img, (454, 340))
    imgBackground = cv2.imread('Resources/background.jpg')
    imgBackground[148:148 + 340, 159:159 + 454] = imgResize
    
    # Get prediction from classifier (call only once)
    prediction = classifier.getPrediction(img)
    classID = prediction[1]

    classDic = {0: None, 1: 0, 2: 0, 3: 3, 4: 3, 5: 1, 6: 1, 7: 2, 8: 2}

    # Overlay waste image and arrow if class is not recyclable
    if classID != 0:
        imgBackground = cvzone.overlayPNG(imgBackground, imgWasteList[classID - 1], (909, 127))
        imgBackground = cvzone.overlayPNG(imgBackground, imgArrow, (978, 320))
        classIDBin = classDic[classID]

        # Overlay bin image using classIDBin
        imgBackground = cvzone.overlayPNG(imgBackground, imgBinsList[classIDBin], (895, 374))

    else:
        # Class is recyclable, handle classIDBin if needed (e.g., logging)
        classIDBin = -1  # Or any default value

    # ... (overlay resized input image, display output)

    # Exit on 'q' press
    if cv2.waitKey(1) == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 73ms/step


In [ ]:
import os
import cvzone
from cvzone.ClassificationModule import Classifier
import cv2

def main():
  # Initialize video capture
  cap = cv2.VideoCapture(0)

  # Load the classifier (replace with your model paths)
  classifier = Classifier('Resources/Model/keras_model.h5', 'Resources/Model/labels.txt')

  # Load the arrow image
  imgArrow = cv2.imread('Resources/arrow.png', cv2.IMREAD_UNCHANGED)
  if imgArrow is None:
      print("Error loading arrow image. Exiting...")
      return

  classIDBin = 0

  # Import all the waste images (with error handling)
  imgWasteList = []
  pathFolderWaste = "Resources/Waste"
  pathList = os.listdir(pathFolderWaste)
  for path in pathList:
      img = cv2.imread(os.path.join(pathFolderWaste, path), cv2.IMREAD_UNCHANGED)
      if img is None:
          print(f"Error loading image: {os.path.join(pathFolderWaste, path)}")
      else:
          imgWasteList.append(img)

  # Import all the bin images (with error handling)
  imgBinsList = []
  pathFolderBins = "Resources/Bins"
  pathList = os.listdir(pathFolderBins)
  for path in pathList:
      img = cv2.imread(os.path.join(pathFolderBins, path), cv2.IMREAD_UNCHANGED)
      if img is None:
          print(f"Error loading image: {os.path.join(pathFolderBins, path)}")
      else:
          imgBinsList.append(img)

  # Define the mapping between classes and bins (handle classID 0)
  classDic = {
      0: -1,  # Default class for missing classID
      1: 0,
      2: 0,
      3: 3,
      4: 3,
      5: 1,
      6: 1,
      7: 2,
      8: 2
  }

  while True:
      _, img = cap.read()
      if img is None:
          print("Error reading camera frame. Exiting...")
          break

      imgResize = cv2.resize(img, (454, 340))  # Resize to match target area dimensions
      imgBackground[148:148 + 340, 159:159 + 454] = imgResize

      imgBackground = cv2.imread('Resources/background.jpg')
      if imgBackground is None:
          print("Error loading background image. Exiting...")
          break

      # Get prediction from the classifier
      prediction = classifier.getPrediction(img)
      classID = prediction[1]
      print(classID)

      if classID != 0:
          if classID - 1 < len(imgWasteList):
              imgBackground = cvzone.overlayPNG(imgBackground, imgWasteList[classID - 1], (909, 127))
              imgBackground = cvzone.overlayPNG(imgBackground, imgArrow, (978, 320))
              classIDBin = classDic[classID]

          if classIDBin < len(imgBinsList):
              imgBackground = cvzone.overlayPNG(imgBackground, imgBinsList[classIDBin], (895, 374))

      imgBackground[148:148 + 340, 159:159 + 454] = imgResize

      # Display images
      cv2.imshow("Image", img)
      cv2.imshow("Output", imgBackground)

      # Exit on 'q' press
      if cv2.waitKey(1) == ord('q'):
          break

  # Release resources
  cap.release()
  cv2.destroyAllWindows()

